In [ ]:
!pip install evaluate

import torch
import evaluate  # Bleu
from transformers import T5ForConditionalGeneration, T5TokenizerFast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
TOKENIZER = T5TokenizerFast.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained('/content/drive/My Drive/pswork/FollowUp', return_dict=True)
Q_LEN = 1024   # Question Length
T_LEN = 256    # Target Length

In [ ]:
def predict_answer(Model, context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).unsqueeze(0)

    outputs = Model.generate(input_ids=input_ids, attention_mask=attention_mask,max_length = 128)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer


In [ ]:
question = "What has been the greatest disappointment in your life?"
answer = ":I didn’t get any high disappoint from my life. I was disappoint\
from myself when I didn’t performed that well in GATE exam and not\
able to get admission in X, but I got admission in Y and now I\
feel that it was okay and not feel disappoint."

In [ ]:
predict_answer(model,question,answer)

'How did you overcome the disappointment?'

In [ ]:
question = "What is the biggest challenge you have faced so far?"
answer = "The biggest challenge that i have faced so far is IIT- JEE\
examination after completing my class 12. I prepared for it for two\
years. Unfortunately I didn’t cracked it."

In [ ]:
predict_answer(model,question,answer)

'How did you overcome the challenge?'

In [ ]:
question = "Short self-presentation: Introduce yourself"
answer = "Hello, I am X, a passionate software engineer. Till my 12th\
standard I studied at my native place itself. My strongest plus point\
according to me is that I dont trust anyone easily and negative point is\
that I cant resist anything wrong happening to anyone around me."

In [ ]:
question= "How would you be an asset to this organisation?"
answer = "I think I have a hunger to learn and solve a problem. For me,\
each problem is a puzzle which I need to solve and I will try till I finish\
it. If I don’t know something, I will learn it and get the job done. My\
hardworking and passionate qualities can be asset to your company."